In [1]:
import pandas as pd
import sys
import numpy as np
from matplotlib import pyplot
import h5py
import os
import sklearn
from sklearn.model_selection import StratifiedShuffleSplit
import torch
import torch.nn as nn
from torch import optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import torch.nn.functional as F
from sklearn import preprocessing
from sklearn.metrics import f1_score
import time

results = []

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#reading in files
X = pd.read_hdf("data/tcga_mutation_train.h5", "expression")
Y = pd.read_hdf('data/tcga_mutation_train.h5', 'labels')

#L1000 subsetting
l1000_file = open("L1000_clueio_genelist.txt")
l1000 = [i.strip() for i in l1000_file.readlines()]
#get l1000 genes that are in data
# L1000_= pd.Series(list(set(X.columns) & set(l1000)))
# X_L1000 = X[L1000_] #subset X data

# Prune expression to only KEGG pathway genes
with open("data/c2.cp.kegg.v6.1.symbols.gmt") as f:
    genes_subset = list(set().union(*[line.strip().split("\t")[2:] for line in f.readlines()]))
X_pruned = X.drop(labels=(set(X.columns) - set(genes_subset)), axis=1, errors="ignore")

#using both kegg and L1000
subset_ = set(X_pruned.columns.tolist() + l1000)
subset = pd.Series(list(set(X.columns) & set(subset_)))

print("Number of genes after subsetting:", len(subset))

X_sub = X[subset] #subset X data


x_array = np.array(X.values, dtype=np.float32)

# extract sample id values
y_names = list(set(Y["detailed_category"].values))
y_names = sorted(y_names)

m,n = X.shape
y_array = np.zeros(shape=(m, len(y_names)+3), dtype=np.float32)

# create a key for id's to indices
y_index_key = {name:i for i,name in enumerate(y_names)}
# generate one-hot vectors for all id's
for m,primary_site_name in enumerate(Y["detailed_category"].values):
    
    index = y_index_key[primary_site_name]
    y_array[m,index] = 1
    
    y_array[m, -3] = Y.iloc[m][6]
    y_array[m, -2] = Y.iloc[m][7]
    y_array[m, -1] = Y.iloc[m][8]
    
    
#split to train + test
xTrain, xTest, yTrain, yTest = sklearn.model_selection.train_test_split(X_sub, y_array, test_size=0.3, random_state=42)
#split test to test and validate 
xTest, xValidate, yTest, yValidate = sklearn.model_selection.train_test_split(xTest, yTest, test_size=1/3, random_state=42)

print(yTrain)
xTrain = np.array(xTrain.values, dtype=np.float32)
xTest = np.array(xTest.values, dtype=np.float32)

Number of genes after subsetting: 5676
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [3]:
class AutoencoderDataset(Dataset):
    def __init__(self, x):
        x_dtype = torch.FloatTensor

        self.length = x.shape[0]

        self.x_data = torch.from_numpy(x).type(x_dtype)

    def __getitem__(self, index):
        return self.x_data[index], self.x_data[index]

    def __len__(self):
        return self.length
    
class PredictorDataset(Dataset):
    def __init__(self, x, y):
        x_dtype = torch.FloatTensor
        y_dtype = torch.FloatTensor
        self.length = x.shape[0]

        self.x_data = torch.from_numpy(x).type(x_dtype)
        self.y_data = torch.from_numpy(y).type(y_dtype)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.length

In [ ]:
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, len(x)), size_average=False)

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD


def train_batch(model, x, y, optimizer, loss_fn):
    # Run forward calculation
    y_predict, avg, stdev = model.forward(x)

    # Compute loss.
    loss = loss_fn(y_predict, y)

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable weights
    # of the model)
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

    return loss.data[0]


def train(model, loader, optimizer, loss_fn, epochs=50):
    losses = list()

    batch_index = 0
    for e in range(epochs):
        for x, y in loader:
            x = Variable(x)
            y = Variable(y)

            loss = train_batch(model=model, x=x, y=y, optimizer=optimizer, loss_fn=loss_fn)
            losses.append(loss)

            batch_index += 1

    model.training = False
    return losses



In [ ]:

def test(model, loader):
    y_vectors = list()
    y_predict_vectors = list()

    batch_index = 0
    for x, y in loader:
        x = Variable(x)
        y = Variable(y)

        y_predict = model.forward(x)

        y_vectors.append(y.data.numpy())
        y_predict_vectors.append(y_predict.data.numpy())

        batch_index += 1

    y_predict_vector = np.concatenate(y_predict_vectors)
    
    return y_predict_vector

def encode(model, loader):
    encoded_vectors = list()
    for x, y in loader:
        x = Variable(x)

        encoded = model.encode(x)

        encoded_vectors.append(encoded.data.numpy())

    encoded_vector = np.concatenate(encoded_vectors)
    return encoded_vector


In [97]:
class VariationalAutoencoder(nn.Module):
    '''
    A simple, general purpose, fully connected network
    '''
    def __init__(self, D_in, size1, size2):
        # Perform initialization of the pytorch superclass
        super(VariationalAutoencoder, self).__init__()
        learning_rate = 1e-2
        
        
        self.encode1 = nn.Linear(D_in, size1)
        self.encodeAvg = nn.Linear(size1, size2)
        self.encodeStdev = nn.Linear(size1, size2)
        self.decode1 = nn.Linear(size2, size1)
        self.decode2 = nn.Linear(size1, D_in)
        
        self.training = True
    
    def forward(self, x):
        '''
        This method defines the network layering and activation functions
        '''
        x = self.encode1(x) # hidden layer
        x = F.relu(x)       # activation function

        avgs = self.encodeAvg(x) # output layer
        stdevs = self.encodeStdev(x)

        if self.training:
            std = torch.exp(stdevs*0.5)
            eps = torch.randn_like(std)
            x = eps.mul(std).add_(avgs)
        else:
            x = avgs
        x = self.decode1(x)
        x = F.relu(x)
        x = self.decode2(x)
        x = F.sigmoid(x)
            
        return x, avgs, stdevs
    
    
    def encode(self, x):
        """Return the trained hidden layer and encoded input"""
        #Get the dataset
        x = self.encode1(x) # hidden layer
        x = F.relu(x)       # activation function

        avg = self.encodeAvg(x)
        return avg

In [100]:

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

def run_logreg(xtrain, ytrain, xtest, ytest, size1, size2, multi):
    
    dataset_train = PredictorDataset(x=xtrain, y=ytrain)
    dataset_test = PredictorDataset(x=xtest, y=ytest)
    dataset_train_encode = AutoencoderDataset(xtrain)
    dataset_test_encode = AutoencoderDataset(xtest)
    
    # Batch size is the number of training examples used to calculate each iteration's gradient
    batch_size_train = 16
    
    data_loader_train = DataLoader(dataset=dataset_train, batch_size=batch_size_train, shuffle=True)
    data_loader_train_encode = DataLoader(dataset=dataset_train_encode, batch_size=batch_size_train, shuffle=True)
    data_loader_test = DataLoader(dataset=dataset_test, batch_size=len(dataset_test), shuffle=False)
    data_loader_test_encode = DataLoader(dataset=dataset_test_encode, batch_size=batch_size_train, shuffle=True)
    
    # Define the hyperparameters
    learning_rate = 1e-2

    # Define the loss function
    loss_fn = nn.MSELoss()  # mean squared error

    # Train and get the resulting loss per iteration
    #Train first autoencoder layer
    encoder_model = VariationalAutoencoder(len(xtrain[0]), size1, size2)

    # Initialize the optimizer with above parameters
    optimizer = optim.SGD(encoder_model.parameters(), lr=learning_rate)
    train(model=encoder_model, loader=data_loader_train_encode, optimizer=optimizer, loss_fn=loss_fn)
    
    newXTrain = encode(encoder_model,data_loader_train_encode)
    newXTest = encode(encoder_model, data_loader_test_encode)
    # Test and get the resulting predicted y values
    aeLogRegModel = OneVsRestClassifier(LogisticRegression()).fit(newXTrain, ytrain)
    
    y_predict_ae = aeLogRegModel.predict(newXTest)
    return y_predict_ae


aeResults = []
yActual = np.array([[y_names[np.argmax(vector[0:-3])], 1 if vector[-3]==1 else 0, 1 if vector[-2]==1 else 0, 1 if vector[-1]==1 else 0] for vector in yTest])
sizes = [(int(len(xTest[0])/100), int(len(xTest[0])/100)), (int(len(xTest[0])*0.75), int(len(xTest[0])/2)), (int(len(xTest[0])/10), int(len(xTest[0])/15))]

'''
print("Results logreg without autoencoder:")
startTime = time.time()
aeLogRegModel = OneVsRestClassifier(LogisticRegression()).fit(xTrain, np.array(yTrain))
yPred = aeLogRegModel.predict(xTest)
elapsedTime = time.time() - startTime
yPred = np.array([[y_names[np.argmax(vector[0:-3])], 1 if vector[-3]>=0.5 else 0, 1 if vector[-2]>=0.5 else 0, 1 if vector[-1]>=0.5 else 0] for vector in yPred])

isCorrect = [[yPred[i][j] == yActual[i][j] for j in range(len(yPred[0]))] for i in range(len(yPred))]


numCorrect = np.sum(isCorrect, axis=0)

pred0, act0 = [v[0] for v in yPred], [v[0] for v in yActual]
pred1, act1 =  [int(v[1]) for v in yPred], [v[1] for v in yActual]
pred2, act2 =  [int(v[2]) for v in yPred], [v[2] for v in yActual]
pred3, act3 =  [int(v[3]) for v in yPred], [v[3] for v in yActual]

##### Write output file and get f1 score
pd.DataFrame({
    "TumorTypePrediction": pred0,
    "TP53MutationPrediction": pred1,
    "KRASMutationPrediction": pred2,
    "BRAFMutationPrediction": pred3,
}).to_csv("test_predictions.tsv", sep="\t")

pd.DataFrame({
    "primary.disease.or.tissue": act0,
    "TP53_mutant": act1,
    "KRAS_mutant": act2,
    "BRAF_mutant": act3,
}).to_csv("test_actuals.tsv", sep="\t")

print("% Correct with logreg : ", (numCorrect/len(isCorrect)))
print("Elapsed time: %f s" % elapsedTime)
!Rscript class/BME230_F1score_V2.R test_predictions.tsv test_actuals.tsv
'''
print()
print("results logreg with autoencoder")
for size in sizes:
    size1, size2 = size
    
    startTime = time.time()
    yPred = run_logreg(xTrain, yTrain, xTest, yTest, size1, size2, True)
    elapsedTime = time.time() - startTime
    
    yPred = np.array([[y_names[np.argmax(vector[0:-3])], 1 if vector[-3]>=0.5 else 0, 1 if vector[-2]>=0.5 else 0, 1 if vector[-1]>=0.5 else 0] for vector in yPred])

    isCorrect = [[yPred[i][j] == yActual[i][j] for j in range(len(yPred[0]))] for i in range(len(yPred))]


    numCorrect = np.sum(isCorrect, axis=0)

    pred0, act0 = [v[0] for v in yPred], [v[0] for v in yActual]
    pred1, act1 =  [int(v[1]) for v in yPred], [v[1] for v in yActual]
    pred2, act2 =  [int(v[2]) for v in yPred], [v[2] for v in yActual]
    pred3, act3 =  [int(v[3]) for v in yPred], [v[3] for v in yActual]

    ##### Write output file and get f1 score
    pd.DataFrame({
        "TumorTypePrediction": pred0,
        "TP53MutationPrediction": pred1,
        "KRASMutationPrediction": pred2,
        "BRAFMutationPrediction": pred3,
    }).to_csv("test_predictions.tsv", sep="\t")

    pd.DataFrame({
        "primary.disease.or.tissue": act0,
        "TP53_mutant": act1,
        "KRAS_mutant": act2,
        "BRAF_mutant": act3,
    }).to_csv("test_actuals.tsv", sep="\t")

    print("% Correct with logreg and autoencoder: ", size1, size2, (numCorrect/len(isCorrect)))
    print("Elapsed time: %fs" % elapsedTime)
    !Rscript class/BME230_F1score_V2.R test_predictions.tsv test_actuals.tsv





results logreg with autoencoder


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:33: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[[ 43.609245  42.332302  17.849098 ... -20.197935  47.171535  57.64376 ]
 [ 45.133675  44.257214  19.395172 ... -20.76133   47.761932  58.92911 ]
 [ 42.214264  41.600704  17.631042 ... -22.246635  47.191742  54.873417]
 ...
 [ 44.112137  43.32668   18.295887 ... -20.394117  46.61047   56.95417 ]
 [ 44.99722   44.28259   19.709044 ... -21.51629   47.64371   58.676586]
 [ 42.438763  41.59054   17.090252 ... -22.740686  46.91838   54.797695]]
[[ 43.769234  43.087967  18.492907 ... -21.341408  47.1753    56.954533]
 [ 46.445415  44.585003  19.936848 ... -20.649244  50.31336   61.256588]
 [ 42.98657   42.599304  18.150915 ... -20.90619   46.743748  57.02777 ]
 ...
 [ 43.688046  43.179302  18.408197 ... -19.658575  46.040817  56.077328]
 [ 43.64888   42.51991   19.536411 ... -19.284359  45.879665  57.135647]
 [ 41.1438    39.99937   17.127672 ... -20.064074  44.107513  54.111458]]
% Correct with logreg and autoencoder:  56 56 [0.01408451 0.69131455 0.94131455 0.92899061]
Elapsed time: 53.390

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:33: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[[ 11.822746  -22.453892    1.3451561 ...   2.2673273   7.1149135
    4.022381 ]
 [ 10.626304  -22.241894    1.7177511 ...   1.7298213   6.9494796
    3.5964346]
 [ 12.149093  -23.664679    2.151298  ...   1.5816927   8.170226
    3.1366534]
 ...
 [ 12.228536  -23.67678     1.2457905 ...   1.3858209   8.772553
    4.2169294]
 [ 12.045946  -22.03976     1.4207411 ...   2.6004062   7.821641
    3.3476887]
 [ 10.930545  -21.635832    1.2464943 ...   2.738262    7.6873684
    4.402821 ]]
[[ 11.244427   -22.941414     2.1550422  ...   1.4575158    7.445836
    4.0086617 ]
 [ 11.511886   -22.835665     2.0770574  ...   0.9462445    7.3714447
    3.575005  ]
 [ 10.771533   -22.090097     1.5245533  ...   1.0035634    7.910443
    3.427771  ]
 ...
 [ 10.181466   -20.889565     1.6641356  ...   2.3839657    7.311992
    3.7379258 ]
 [ 10.8505125  -22.22744      1.5683513  ...   0.98852885   7.547903
    3.5182896 ]
 [ 12.731482   -19.8488       1.2632952  ...  -0.03617124   7.402346
    3.05614

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:33: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[[-48.339447   9.480579  31.687338 ... -47.461308  33.4022   -11.69935 ]
 [-45.00673    9.562611  30.68357  ... -45.23989   31.38032  -10.079558]
 [-49.11628    8.180693  31.964169 ... -49.212383  33.816303 -11.609059]
 ...
 [-48.22629    9.070653  31.705486 ... -48.50416   33.956554 -11.285165]
 [-48.402885  10.218317  32.195633 ... -48.171852  33.910374 -12.186844]
 [-48.9361    10.406243  31.47539  ... -48.611446  33.236168 -11.881174]]
[[-49.55973     9.076023   31.791264  ... -49.981194   33.879982
  -12.875681 ]
 [-51.221573    9.43601    33.851303  ... -50.954205   34.820923
  -12.266271 ]
 [-52.080593   10.457656   34.060135  ... -52.826122   36.903217
  -12.222719 ]
 ...
 [-51.434452    8.923946   33.483887  ... -51.35469    35.245235
  -12.18439  ]
 [-49.728653   10.213206   32.261322  ... -50.191597   34.098366
  -12.903389 ]
 [-51.55493    10.0553055  33.572117  ... -51.21119    35.841896
  -12.731243 ]]
% Correct with logreg and autoencoder:  567 378 [0.01584507 0.65962441